# Heston Almost Exact Simulation

In this notebook we adapt the implementation of Heston Almost Exact Simulation by [Nicholas Burgess](https://github.com/nburgessx/Papers/tree/main/HestonSimulation), into a qablet MC Model, and use it to price

- Vanilla Options
- Forward Starting Options
- Accumulator Cliquets

In [ ]:
from datetime import datetime

import numpy as np
import pandas as pd
from qablet_contracts.eq.cliquet import Accumulator
from qablet_contracts.eq.forward import ForwardOption
from qablet_contracts.eq.vanilla import Option
from qablet_contracts.timetable import py_to_ts

from src.models.aes import HestonAESMC
from src.qablet_utils import MCPricer

## Create Dataset
Create the dataset, with MC params, discounts and fwds as required by the [Dataset API](https://qablet-academy.github.io/intro/dataset/) 

For the `HESTON` section, add components as needed by the methods defined above. 

In [ ]:
times = np.array([0.0, 5.0])
rates = np.array([0.1, 0.1])
discount_data = ("ZERO_RATES", np.column_stack((times, rates)))

ticker = "EQ"
spot = 100.0
div_rate = 0.0
fwds = spot * np.exp((rates - div_rate) * times)
fwd_data = ("FORWARDS", np.column_stack((times, fwds)))

dataset = {
    "BASE": "USD",
    "PRICING_TS": py_to_ts(datetime(2023, 12, 31)).value,
    "ASSETS": {"USD": discount_data, ticker: fwd_data},
    "MC": {
        "PATHS": 2_500,
        "TIMESTEP": 1 / 1000,
        "SEED": 1,
    },
    "HESTON": {
        "ASSET": ticker,
        "INITIAL_VAR": 0.04,
        "LONG_VAR": 0.04,
        "VOL_OF_VOL": 1.0,
        "MEANREV": 0.5,
        "CORRELATION": -0.9,
    },
}

## Calculate Single Option Price

Define a [Vanilla Option](https://qablet.github.io/qablet-contracts/examples/equity_vanilla/) and calculate price.

In [ ]:
model = MCPricer(HestonAESMC)

# Create Contract
strike = 100
ticker = "EQ"
timetable = Option(
    "USD", ticker, strike=strike, maturity=datetime(2024, 12, 31), is_call=True
).timetable()
print(timetable["events"].to_pandas())

price, stats = model.price(timetable, dataset)
print(f"price: {price:11.6f}")

                       time op  quantity unit track
0 2024-12-31 00:00:00+00:00  >       0.0  USD      
1 2024-12-31 00:00:00+00:00  +    -100.0  USD      
2 2024-12-31 00:00:00+00:00  +       1.0   EQ      
price:   12.100248


## Forward Starting Option

Define a [Forward Starting Option](https://qablet.github.io/qablet-contracts/examples/equity_fwdoption/) and calculate price.

In [ ]:
for strike_date in [
    datetime(2024, 3, 31),
    datetime(2024, 6, 30),
    datetime(2024, 11, 30),
]:
    timetable = ForwardOption(
        "USD",
        ticker,
        strike_rate=1.0,
        strike_date=strike_date,
        maturity=datetime(2024, 12, 31),
        is_call=True,
    ).timetable()
    price, stats = model.price(timetable, dataset)
    print(f"strike_date: {strike_date.date()} price: {price:11.6f}")

strike_date: 2024-03-31 price:    9.048823
strike_date: 2024-06-30 price:    6.308333
strike_date: 2024-11-30 price:    1.362673


## Accumulator Cliquet

Define instances of [Accumulator](https://qablet.github.io/qablet-contracts/examples/equity_cliquet/) and calculate price.

In [ ]:
# Create the cliquet, quarterly fixing dates
fix_dates = pd.bdate_range(
    datetime(2023, 12, 31), datetime(2024, 12, 31), freq="1BQE"
)

for local_cap in [0.02, 0.04, 0.06]:
    global_floor = 0.0
    local_floor = -local_cap
    timetable = Accumulator(
        "USD",
        ticker,
        fix_dates,
        global_floor,
        local_floor,
        local_cap,
        state={"S_PREV": 1.0},
    ).timetable()

    price, stats = model.price(timetable, dataset)
    print(f"cap/floor: {local_cap} price: {price:11.6f}")

cap/floor: 0.02 price:    4.721853
cap/floor: 0.04 price:    8.150092
cap/floor: 0.06 price:   10.183573
